### Creation of the mashup datasets

The collection of the data results in ten different datasets:
- <b>Scholarships</b>: one per year, they contain all the information about "diritto allo studio" (see terminological caveat in the main documentation). Each funding is divided according to the tipology of the expenditure (scholarship, international mobility, canteens, dorms etc). They are saved in the repository as [<code>/data/source/dsu2016.csv</code>](../../data/source/dsu2016.csv), [<code>/data/source/dsu2017.csv</code>](../../data/source/dsu2017.csv), [<code>/data/source/dsu2018.csv</code>](../../data/source/dsu2018.csv) and [<code>/data/source/dsu2019.csv</code>](../../data/source/dsu2019.csv);
- <b>Fees</b>: one per year, they contain information all the information about fees. They divide between average fee calculated among all the enrolled students (included free-waiver) and all the actually paying ones. Only the latter has been considered for the analysis. They are saved in the repository as [<code>/data/source/fees2016.csv</code>](../../data/source/fees2016.csv), [<code>/data/source/fees2017.csv</code>](../../data/source/fees2017.csv), [<code>/data/source/fees2018.csv</code>](../../data/source/fees2018.csv), [<code>/data/source/fees2019.csv</code>](../../data/source/fees2019.csv);
- <b>Total students</b>: one single table ([<code>/data/source/student.csv</code>](../../data/source/student.csv)) containing all the information about the total amount of enrolled students, divided per gender and academic year;
- <b>International students</b>: one single table ([<code>/data/source/intStudent.csv</code>](../../data/source/intStudent.csv)) containing all the information about the total amount of enrolled students, divided per academic year;

It has been decided to maintain the subdivision on a chronological basis for the output datasets: the main goal of the analysis is hence to analyse possible correlations among the variables. Description of the trends across reference period (academic years 2015/16, 2016/17, 2017/18, 2018/19) is a secondary goal. The algorithm should compress the information in four output csv files. The following picture visualizes syntetically the process, which also comported the implementation of simple computations to convert absolute measurements (e.g. number of international students) into relative (their percentage over the entirety of student population)

<img src="../../assets/img/mashupGraph.png" alt="Mashup pipeline graph">
<br><br><small>The visualization was created through the software Draw.io</small>

The algorithm can be executed at the file [<code>mashup.py</code>](../mashup.py). Its main function relies on two ancillary functions. The first one (<code>mergeCount</code>) allows to summarize into a single value all the information which are presented as divided in the main output. This function is called to compress expenditure information about scholarships and to remove the sex-based distinction in student enrollment.

In [1]:
import pandas as pd

def mergeCount(sourceDf, destDf, col1, col2, col3, idcol, namecol, valcol):
    i = 0
    for idx, row in sourceDf.iterrows():
        destDf.at[i, col1] = row[idcol]  
        destDf.at[i, col2] = row[namecol]
        temp = sourceDf.query('{0} == "{1}"'.format(idcol, row[idcol]))
        sum = 0
        for tempidx, temprow in temp.iterrows():
            if "," in str(temprow[valcol]):
                temprow[valcol] = str(temprow[valcol]).replace(",", ".")
            sum += int(float(temprow[valcol]))
        destDf.at[i, col3] = sum
        i += 1
    destDf.drop_duplicates(ignore_index=True, inplace=True)
    return destDf

The merging process relied on the internal id assigned in the source datasets. The attribution of these codes, however, is not coherent across al the datasets. Only the first two (scholarships and fees) distinguished among the different seats of the same institutions with the adjunction of two digits at the end: for example, Politecnico di Torino has code 102, while its Information Engineering Departmente, seated in Verres, has code 10202. A comparative analysis of the id suggested that it could be implemented an algorithm which transform the ID into a 6-digits value and the cut it at the first four (starting from the left). This operation will be implemented directly in the body of the main function. The last two datasets (students and international students) are instead handled by an ancillary function, <code>refineStudId(myDf)</code>,which adds (if needed) digits to reach the desired length (i.e. 4 digits):

In [2]:
def refineStudId(myDf):
    idList = [] 
    for idx, row in myDf.iterrows():
        while len(str(row["AteneoCOD"])) < 4:
            row["AteneoCOD"] = str(row["AteneoCOD"]) + "0"
        idList.append(str(row["AteneoCOD"]))
    myDf["AteneoCOD"] = idList
    return myDf

The main function <code>dataligner(dsupath, feepath, studpath, intpath, year, outputpath)</code> translates in Python syntax the process described in the previous chart. See the inline comments for an essential step-to-step description of the algorithm. 

In [3]:
def dataligner(dsupath, feepath, studpath, intpath, year, outputpath):
    
    #STEP 1
    #Creates source dataframes from csv
    dsuRaw = pd.read_csv(dsupath, sep=";", encoding = "ISO-8859-1")
    feeRaw = pd.read_csv(feepath, sep=";", encoding = "ISO-8859-1")
    studentRaw = pd.read_csv(studpath, sep=",", encoding = "ISO-8859-1").query('AnnoA == "{0}"'.format(year))
    intstudRaw = pd.read_csv(intpath, sep=",", encoding = "ISO-8859-1").query('AnnoA == "{0}"'.format(year))
    
    #Defines two intermediate and one last output dataframes
    dsuDf = pd.DataFrame(columns=["uni_id", "university", "scholarship"])
    studDf = pd.DataFrame(columns=["uni_id", "uni", "total_students"])
    outputDf = pd.DataFrame(columns=["uni_id", "uni", "total_students", "int_students", "perc_intern", "relative_scholarship"])

    #STEP 2
    #Clean dataframe about "Diritto allo studio", aka scholarships
    dsuRaw = dsuRaw.query('TIPO_ISTITUTO == "Ateneo"')
    dsuRaw = dsuRaw.astype({'NOME_ISTITUTO':'string'})
    dsuRaw = dsuRaw.astype({'CODICE_ISTITUTO':'int64'})
    dsuRaw = dsuRaw.astype({'CODICE_ISTITUTO':'string'})

    #Do not consider telematic university, simplify redundant records, uniform the ids
    for idx, row in dsuRaw.iterrows():
        if "telematic" not in row['NOME_ISTITUTO']:
            if "<" in row[6]:
                row[6] = (row[6].split("<"))[0]
            elif "(" in row[6]:
                row[6] = (row[6].split("("))[0]
            row[5]= str(row[5])
            while len(str(row[5])) < 6:
                row[5] = str(str(row[5]) + "0")
            dsuRaw.at[idx, "CODICE_ISTITUTO"] = row[5][:-2:]       
    dsuDf = mergeCount(dsuRaw, dsuDf, "uni_id", "university", "scholarship", "CODICE_ISTITUTO", "NOME_ISTITUTO", "SPESA_LAUREA")

    #STEP 3
    #Refine and uniform the ids in the fees dataframe
    for idx, row in feeRaw.iterrows():
        while str(row["COD_Ateneo"]).startswith("0"):
            row["COD_Ateneo"] = str(row["COD_Ateneo"])[1:]
        while len(str(row["COD_Ateneo"])) < 4:
            row["COD_Ateneo"] = str(row["COD_Ateneo"]) + "0"
        feeRaw.at[idx, "COD_Ateneo"] = row["COD_Ateneo"]

    #Merge scholarships and fees
    dsuDf = pd.merge(dsuDf, feeRaw, how='inner',left_on='uni_id',right_on='COD_Ateneo')
    dsuDf = dsuDf[['uni_id', 'NOME_ATENEO', 'scholarship', 'TASSA_MEDIA_PAGANTI_LAUREA', 'TASSA_MEDIA_TOTALE_ISCRITTI_LAUREA']].rename(columns= {'NOME_ATENEO':'uni', 'TASSA_MEDIA_PAGANTI_LAUREA':'paidfee','TASSA_MEDIA_TOTALE_ISCRITTI_LAUREA':'totalfee'})
    dsuDf.drop_duplicates(ignore_index=True, inplace=True)

    #STEP 4
    #Apply similar processes for overall enrolling
    studentRaw = refineStudId(studentRaw)
    studDf = mergeCount(studentRaw, studDf, "uni_id", "uni", "total_students", "AteneoCOD", "AteneoNOME", "Isc")
    studDf = pd.merge(dsuDf, studDf, how='inner',left_on='uni_id',right_on='uni_id')
    studDf = studDf[['uni_id', 'uni_y', 'scholarship', 'paidfee', 'totalfee', 'total_students']].rename(columns= {'uni_y':'uni'})

    #Apply similar processes for international enrolling
    intstudRaw = refineStudId(intstudRaw)
    outputDf = mergeCount(intstudRaw, outputDf, "uni_id", "uni", "int_students", "AteneoCOD", "AteneoNOME", "Isc_S") 
    outputDf = pd.merge(outputDf, studDf, how='inner',left_on='uni_id',right_on='uni_id')
    outputDf = outputDf[['uni_id', 'uni_y', 'scholarship', 'paidfee', 'totalfee', 'total_students_y', 'int_students', 'perc_intern', 'relative_scholarship']].rename(columns= {'uni_y':'uni','total_students_y':'total_students'})

    #Compute percentages of international students
    for idx, row in outputDf.iterrows():
        outputDf.at[idx, "perc_intern"] = 100 * int(row["int_students"])/int(row["total_students"])
        
    #Compute relative scholarship per students
    for idx, row in outputDf.iterrows():
        outputDf.at[idx, "relative_scholarship"] = int(row["scholarship"])/int(row["total_students"])
    
    #Replace comma with dot to streamline further data conversion 
    for idx, row in outputDf.iterrows(): 
        val1 = str(row["paidfee"]); val2 = str(row["totalfee"])
        outputDf.at[idx, "paidfee"] = val1.replace(',', '.')
        outputDf.at[idx, "totalfee"] = val2.replace(',', '.')
    outputDf['paidfee'] = outputDf['paidfee'].astype(float)
    outputDf['totalfee'] = outputDf['totalfee'].astype(float)
    
    #Some telematic univeristies remain in the dF: we have to drop these institutions
    outputDf.drop(outputDf[outputDf['uni'].str.contains(" - telematica")].index, inplace = True)

    #STEP 6
    #Export the dataframe
    outputDf.to_csv(outputpath, index=False)
    return outputDf

To improve the readablity of the script, filepaths are stored as string variables and used as input of the main function, where the new files are saved through the method <code>pandas.DataFrame.to_csv(filepath)</code> before being returned.

In [5]:
stud = "../../data/source/student.csv"
ints = "../../data/source/intStudent.csv"

dsu2016 = "../../data/source/dsu2016.csv"
fee2016 = "../../data/source/fees2016.csv"
dest2016 = "../../data/output/2016.csv"

print(dataligner(dsu2016, fee2016, stud, ints, "2015/2016", dest2016))

   uni_id                        uni scholarship   paidfee  totalfee  \
0    1010                     Torino    14573831   1374.95   1275.10   
1    1020         Torino Politecnico    14180917   1093.96    948.63   
2    2010         Piemonte Orientale     2045905   1314.37   1230.55   
3    4010  Bra Scienze Gastronomiche           0  10307.32   8798.04   
4    7010                      Aosta      894297   1356.28   1187.87   
..    ...                        ...         ...       ...       ...   
73   8301                    Messina     4334128   1022.82   1009.29   
74   8601                  Enna KORE     1549138   2760.04   2760.04   
75   8701                    Catania     8991901    741.29    663.47   
76   9001                    Sassari     5660474    879.94    737.18   
77   9201                   Cagliari    18176242    795.16    641.48   

   total_students int_students perc_intern relative_scholarship  
0           66350         3708    5.588546           219.650806  
1  

In [6]:
dsu2017 = "../../data/source/dsu2017.csv"
fee2017 = "../../data/source/fees2017.csv"
dest2017 = "../../data/output/2017.csv"

print(dataligner(dsu2017, fee2017, stud, ints, "2016/2017", dest2017))

   uni_id                        uni scholarship   paidfee  totalfee  \
0    1010                     Torino    18383953   1344.67   1223.25   
1    1020         Torino Politecnico    15465478   1086.65    919.64   
2    2010         Piemonte Orientale     2570632   1326.97   1211.31   
3    4010  Bra Scienze Gastronomiche           0  10060.42   8832.71   
4    7010                      Aosta      837365   1623.70   1351.72   
..    ...                        ...         ...       ...       ...   
79   8301                    Messina     4801688   1107.77    924.26   
80   8601                  Enna KORE     1926643   2893.28   2893.28   
81   8701                    Catania    12872273    896.43    769.59   
82   9001                    Sassari     8994301    890.09    685.29   
83   9201                   Cagliari    20919445    801.50    594.10   

   total_students int_students perc_intern relative_scholarship  
0           68422         3871    5.657537           268.684824  
1  

In [7]:
dsu2018 = "../../data/source/dsu2018.csv"
fee2018 = "../../data/source/fees2018.csv"
dest2018 = "../../data/output/2018.csv"

print(dataligner(dsu2018, fee2018, stud, ints, "2017/2018", dest2018))

   uni_id                        uni scholarship   paidfee  totalfee  \
0    1010                     Torino    20972522   1502.08   1224.67   
1    1020         Torino Politecnico    15307754   1147.08    884.24   
2    2010         Piemonte Orientale     3076591   1401.60   1147.79   
3    4010  Bra Scienze Gastronomiche           0  10311.24   9283.23   
4    7010                      Aosta      692508   1667.51   1387.92   
..    ...                        ...         ...       ...       ...   
81   8301                    Messina    12181696   1262.84    844.93   
82   8601                  Enna KORE     1594466   3034.75   3034.15   
83   8701                    Catania     8365996    770.19    559.05   
84   9001                    Sassari     9121637    863.49    581.44   
85   9201                   Cagliari    13585559    900.99    592.36   

   total_students int_students perc_intern relative_scholarship  
0           71597         4366    6.098021           292.924592  
1  

In [8]:
dsu2019 = "../../data/source/dsu2019.csv"
fee2019 = "../../data/source/fees2019.csv"
dest2019 = "../../data/output/2019.csv"

print(dataligner(dsu2019, fee2019, stud, ints, "2018/2019", dest2019))

   uni_id                        uni scholarship   paidfee  totalfee  \
0    1010                     Torino    21556622   1430.90   1150.91   
1    1020         Torino Politecnico    15548115   1239.56    829.62   
2    2010         Piemonte Orientale     3087494   1424.04   1155.70   
3    4010  Bra Scienze Gastronomiche        8028  10341.61   9275.77   
4    7010                      Aosta      997232   1671.69   1383.35   
..    ...                        ...         ...       ...       ...   
82   8301                    Messina     7040031   1246.60    820.13   
83   8601                  Enna KORE     1516254   3089.71   3088.41   
84   8701                    Catania     9135199    791.20    549.74   
85   9001                    Sassari    13083381    906.92    543.68   
86   9201                   Cagliari    26527950   1013.66    526.92   

   total_students int_students perc_intern relative_scholarship  
0           72877         4493    6.165182           295.794585  
1  